In [ ]:
!pip install geopandas
!pip install folium matplotlib mapclassify

In [ ]:
src_path = '/content/drive/MyDrive/sk쉴더스/프로젝트1/데이터/'

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely import wkt

In [ ]:
def point_to_grid(grid_df, point_df, col_name):
  '''
  ## Args
  `grid_df` 격자 geopandas 데이터프레임 5179
  `point_df` 격자를 매칭할 geopandas 데이터프레임 5179

  ## Returns
  `count_by_grid` 격자별 개수
  '''
  match_df = gpd.sjoin(point_df, grid_df, how='left')
  match_df = match_df[~match_df['gid'].isnull()]
  match_by_grid = match_df.pivot_table(index='gid', values=match_df.columns[0], aggfunc='count')
# --------
  count_by_grid = pd.DataFrame()
  count_by_grid['gid'] = grid_df['gid']
  count_by_grid[col_name] = 0

  for row in match_by_grid.iterrows():
    gid = row[0]
    count = row[1][match_df.columns[0]]
    count_by_grid.loc[count_by_grid['gid']==gid, col_name] = count

  return count_by_grid

In [ ]:
grid = pd.read_csv(src_path + 'unique_grid.csv')

In [ ]:
danran = pd.read_csv(src_path + '음식점/서울시 강남구 단란주점영업 인허가 정보.csv', encoding='cp949')
uheung = pd.read_csv(src_path + '음식점/서울시 강남구 유흥주점영업 인허가 정보.csv', encoding='cp949')
normal = pd.read_csv(src_path + '음식점/서울시 강남구 일반음식점 인허가 정보.csv', encoding='utf-8')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (27,36,41,42,46,48,56,57,59,61,62,63,66,69,70,71,80) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
danran = danran[~(danran['좌표정보(X)'].isna() | danran['좌표정보(Y)'].isna())]

In [ ]:
uheung = uheung[~(uheung['좌표정보(X)'].isna() | uheung['좌표정보(Y)'].isna())]

In [ ]:
normal = normal[~(normal['좌표정보(X)'].isna() | normal['좌표정보(Y)'].isna())]

In [ ]:
danran = danran[danran['영업상태명']=='영업/정상'][['관리번호','좌표정보(X)','좌표정보(Y)']]
uheung = uheung[uheung['영업상태명']=='영업/정상'][['관리번호','좌표정보(X)','좌표정보(Y)']]
normal = normal[normal['영업상태명']=='영업/정상'][['관리번호','좌표정보(X)','좌표정보(Y)']]

In [ ]:
grid['geometry'] = grid['geometry'].apply(wkt.loads)
geo_grid = gpd.GeoDataFrame(grid, crs='epsg:5179')

In [ ]:
geo_danran = gpd.GeoDataFrame(
    danran, geometry=gpd.points_from_xy(danran['좌표정보(X)'], danran['좌표정보(Y)'])
)
geo_danran.set_crs(epsg=5174, inplace=True)
geo_danran.to_crs(epsg=5179, inplace=True)
geo_danran.head(2)

,관리번호,좌표정보(X),좌표정보(Y),geometry
1,3220000-103-1983-04059,202372.465591,446641.566132,POINT (958258.969 1947072.517)
6,3220000-103-1991-03129,202878.095900,445870.979863,POINT (958760.316 1946299.561)


In [ ]:
geo_uheung = gpd.GeoDataFrame(
    uheung, geometry=gpd.points_from_xy(uheung['좌표정보(X)'], uheung['좌표정보(Y)'])
)
geo_uheung.set_crs(epsg=5174, inplace=True)
geo_uheung.to_crs(epsg=5179, inplace=True)
geo_uheung.head(2)

,관리번호,좌표정보(X),좌표정보(Y),geometry
2,3220000-102-1976-04321,201864.398628,446138.700258,POINT (957748.442 1946572.543)
26,3220000-102-1980-04353,204377.539372,445582.398728,POINT (960257.628 1946003.155)


In [ ]:
geo_normal = gpd.GeoDataFrame(
    normal, geometry=gpd.points_from_xy(normal['좌표정보(X)'], normal['좌표정보(Y)'])
)
geo_normal.set_crs(epsg=5174, inplace=True)
geo_normal.to_crs(epsg=5179, inplace=True)
geo_normal.head(2)

,관리번호,좌표정보(X),좌표정보(Y),geometry
0,3220000-101-1007-00001,203520.8811,446950.2856,POINT (959408.559 1947375.029)
7,3220000-101-1899-20451,205130.5917,445590.0968,POINT (961010.417 1946006.862)


In [ ]:
d1 = point_to_grid(geo_grid, geo_danran, '단란주점')

In [ ]:
d2 = point_to_grid(geo_grid, geo_uheung, '유흥주점')

In [ ]:
d3 = point_to_grid(geo_grid, geo_normal, '일반음식점')

In [ ]:
df = pd.merge(d1, d2, on='gid')
df = pd.merge(df, d3, on='gid')

In [ ]:
df.to_csv(src_path + 'restaurant.csv', index=False)

In [ ]:
geo_normal[geo_normal['관리번호']=='3220000-101-1007-00001'].explore(tiles='http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}', attr='Google')

In [ ]:
geo_uheung.explore()

In [ ]:
geo_danran.explore()